In [1]:
print("hello, world!")

hello, world!


In [2]:
from graphql import build_schema, parse

In [ ]:
from graphql import build_schema, parse

with open('../assets/schemas/opensea/opensea_original_schema.graphql', 'r') as schema_file:
    schema_str = schema_file.read()

# TEMP: Try to parse the schema given the errors we are seeing
custom_definitions = '''
scalar BigDecimal
scalar BigInt

directive @entity on OBJECT
directive @dailySnapshot on OBJECT
directive @regularPolling on OBJECT
directive @derivedFrom on OBJECT
directive @transaction on OBJECT
'''

full_schema_str = custom_definitions + schema_str
schema = build_schema(full_schema_str)

for type_name, graphql_type in schema.type_map.items():
    if type_name.startswith('__'):
        continue  # Skip introspection types
    print(f"Type: {type_name}")
    for field_name, field in graphql_type.fields.items():
        print(f"  Field: {field_name} (type: {field.type})")

TypeError: Directive '@derivedFrom' may not be used on field definition.

Unknown argument 'field' on directive '@derivedFrom'.